[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/vizier/blob/main/docs/guides/developer/writing_algorithms.ipynb)

# Writing Algorithms and Pythia Policies
This documentation will allow a developer to:

*   Understand the basic structure of a Pythia policy.
*   Use the Designer API for simplfying algorithm design.




## Installation and reference imports

In [ ]:
!pip install google-vizier

In [ ]:
from typing import Optional, Sequence

from vizier import pythia
from vizier import pyvizier
from vizier import algorithms
from vizier._src.algorithms.policies import designer_policy
from vizier._src.algorithms.evolution import nsga2

## Pythia Policies
The Pythia Service maps algorithm names to `Policy` objects. All algorithms which need to be hosted on the server must eventually be wrapped into a `Policy`.

Every `Policy` is injected with a `PolicySupporter`, which is a client used for fetching data from the datastore. This design choice serves two core purposes:

1. The `Policy` is effectively stateless, and thus can be deleted and recovered at any time (e.g. due to a server preemption or failure).
2. Consequently, this avoids needing to save an explicit and potentially complicated algorithm state. Instead, the "algorithm state" can be recovered purely from the entire study containing (`metadata`, `study_config`, `trials`).

We show the `Policy` abstract class explicitly below. Exact class entrypoint can be found [here](https://github.com/google/vizier/blob/main/vizier/pythia.py).

In [ ]:
class Policy(abc.ABC):
  """Interface for Pythia2 Policy subclasses."""

  @abc.abstractmethod
  def suggest(self, request: SuggestRequest) -> SuggestDecision:
    """Compute suggestions that Vizier will eventually hand to the user."""

  @abc.abstractmethod
  def early_stop(self, request: EarlyStopRequest) -> EarlyStopDecisions:
    """Decide which Trials Vizier should stop."""

  @property
  def should_be_cached(self) -> bool:
    """Returns True if it's safe & worthwhile to cache this Policy in RAM."""
    return False

### Fundamental Rule of Service Pythia Policies
For algorithms used in the Pythia Service, the fundamental rule is to assume that a Pythia policy class instance will only call once per user interaction:
*   `__init__`
*   `suggest()`

and be immediately deleted afterwards.

## Example Pythia Policy
Here, we write a toy policy, where we only act on `CATEGORICAL` parameters for simplicity. The `make_parameters` function will simply for-loop over every category and then cycle back.

In [ ]:
def make_parameters(study_config: pyvizier.StudyConfig,
                    index: int) -> pyvizier.ParameterDict:
  parameter_dict = pyvizier.ParameterDict()
  for parameter_config in study_config.search_space.parameters:
    if parameter_config.type == pyvizier.ParamterType.CATEGORICAL:
      feasible_values = parameter_config.feasible_values
      categorical_size = len(feasible_values)
      parameter_dict[parameter_config.name] = pyvizier.ParameterValue(
          value=feasible_values[index % categorical_size])
    else:
      raise ValueError("This function only supports CATEGORICAL parameters.")
  return parameter_dict

To collect the `index` from the database, we will use the `PolicySupporter` to obtain all completed trials and look at the maximum trial ID.

In [ ]:
def get_next_index(policy_supporter: pythia.PolicySupporter):
  """Returns current trial index."""
  completed_trial_ids = [
      t.id for t in policy_supporter.GetTrials(
          status_matches=pyvizier.TrialStatus.COMPLETED)
  ]

  if completed_trial_ids:
    return max(completed_trial_ids)
  return 0

We can now put it all together into our Pythia Policy.

In [ ]:
class MyPolicy(pythia.Policy):
  def __init__(self, policy_supporter: pythia.PolicySupporter):
    self._policy_supporter = policy_supporter

  def suggest(self, request: pythia.SuggestRequest) -> pythia.SuggestDecision:
    """Gets number of Trials to propose, and produces Trials."""
    suggest_decision_list = []
    for _ in range(request.count):
      index = get_next_index(self._policy_supporter)
      parameters = make_parameters(request.study_config, index)
      suggest_decision_list.append(
          pyvizier.TrialSuggestion(parameters=parameters))
    return pythia.SuggestDecision(
        suggestions=suggest_decision_list, metadata=pyvizier.MetadataDelta())

## Designers
While Pythia policies are the proper interface for hosting algorithms on
the server, we also provide the `Designer` API to simplify algorithm development and avoid distributed environment logic. A `Designer` locally performs a standard suggest-update loop in RAM, during the lifetime of a study.

We display the `Designer` class below. Exact class entrypoint can be found
[here](https://github.com/google/vizier/blob/main/vizier/algorithms/__init__.py).

In [ ]:
class Designer(...):
  """Suggestion algorithm for sequential usage."""

  @abc.abstractmethod
  def update(self, delta: CompletedTrials) -> None:
    """Reflect the delta in the designer's state."""

  @abc.abstractmethod
  def suggest(self,
              count: Optional[int] = None) -> Sequence[vz.TrialSuggestion]:
    """Make new suggestions."""

To implement our same algorithm above in a Designer, only the `update()` and `suggest()` methods need to be implemented using our previous `make_parameters` function. The designer class can now store completed trials inside its `self._completed_trials` attribute.

In [ ]:
class MyDesigner(algorithms.Designer):

  def __init__(self, study_config: pyvizier.StudyConfig):
    self._study_config = study_config
    self._completed_trials = []

  def update(self, delta: algorithms.CompletedTrials) -> None:
    self._completed_trials.extend(delta.completed)

  def suggest(
      self, count: Optional[int] = None) -> Sequence[pyvizier.TrialSuggestion]:
    if count is None:
      return []
    completed_trial_ids = [t.id for t in self._completed_trials]
    current_index = max(completed_trial_ids)
    return [
        make_parameters(self._study_config, current_index + i)
        for i in range(count)
    ]

## Wrapping a `Designer` to a Pythia `Policy`
Note that in the above implementation of `MyDesigner`, the entire algorithm (if deleted or preempted) can conveniently be recovered in just a **single** call of `update()` after `__init__`.

Thus we may immediately wrap `MyDesigner` into a Pythia Policy with the following Pythia `suggest()` implementation:

*   Create the designer temporarily.
*   Update the temporary designer with **all** previously completed trials.
*   Obtain suggestions from the temporary designer.

This is done conveniently with the `DesignerPolicy` wrapper ([code](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/policies/designer_policy.py)):

In [ ]:
class DesignerPolicy(pythia.Policy):
  """Wraps a Designer into a Pythia Policy."""

  def __init__(self, supporter: pythia.PolicySupporter,
               designer_factory: Factory[vza.Designer]):
    self._supporter = supporter
    self._designer_factory = designer_factory

  def suggest(self, request: pythia.SuggestRequest) -> pythia.SuggestDecision:
    designer = self._designer_factory(request.study_config)
    new_trials = self._supporter.GetTrials(
        status_matches=vz.TrialStatus.COMPLETED)
    designer.update(vza.CompletedTrials(new_trials))

    return pythia.SuggestDecision(
        designer.suggest(request.count), metadata=vz.MetadataDelta())

Below is the actual act of wrapping:

In [ ]:
designer_factory = lambda study_config: MyDesigner(study_config)
supporter: pythia.PolicySupporter = ... # Assume PolicySupporter was created.
pythia_policy = designer_policy.DesignerPolicy(
    supporter=supporter, designer_factory=designer_factory)

## Serializing algorithm states
The above method can gradually become slower as the number of completed trials in the study increases. Furthermore, it does not account for algorithms which store information about non-completed suggestions as well.

Thus we may consider storing a compressed representation of the algorithm state instead. Examples include:

*   The coordinate position in a grid search algorithm.
*   The population for evolutionary algorithms such as NSGA2.
*   Directory location for stored neural network weights.

As a simple example, consider the case if our designer needed to store a `_counter` of **all** (both completed and non-completed) suggestions it has made:

In [ ]:
class CounterDesigner(algorithms.Designer):

  def __init__(self, ...):
    ...
    self._counter = 0

  def suggest(
      self, count: Optional[int] = None) -> Sequence[pyvizier.TrialSuggestion]:
    ...
    self._counter += len(suggestions)
    return suggestions

Vizier offers
[two Designer subclasses](https://github.com/google/vizier/blob/main/vizier/interfaces/serializable.py), both of which will use the `Metadata` primitive to store algorithm state data:

*   `SerializableDesigner` will use additional `recover`/`dump` methods and should be used if the entire algorithm state can be easily serialized and can be saved and restored in full.
*   `PartiallySerializableDesigner` will use additional `load`/`dump` methods and be used if the algorithm has subcomponents that are not easily serializable. State recovery will be handled by calling the Designer's `__init__` (with same arguments as before) and then `load`.

They can also be converted into Pythia Policies using `SerializableDesignerPolicy` and `PartiallySerializableDesignerPolicy` respectively.

Below is an example modifying our `CounterDesigner` into `CounterSerialDesigner` and `CounterPartialDesigner` respectively:

In [ ]:
class CounterSerialDesigner(algorithms.SerializableDesigner):

  def __init__(self, counter: int):
    self._counter = counter

  @classmethod
  def recover(cls, metadata: pyvizier.Metadata) -> CounterSerialDesigner:
    return cls(metadata['counter'])

  def dump(self) -> pyvizier.Metadata:
    metadata = pyvizier.Metadata()
    metadata['counter'] = str(self._counter)
    return metadata


class CounterPartialDesigner(algorithms.PartiallySerializableDesigner):

  def load(self, metadata: pyvizier.Metadata) -> None:
    self._counter = int(metadata['counter'])

  def dump(self) -> pyvizier.Metadata:
    metadata = pyvizier.Metadata()
    metadata['counter'] = str(self._counter)
    return metadata

## Additional References
*   Our [policies folder](https://github.com/google/vizier/tree/main/vizier/_src/algorithms/policies) contains examples of Pythia policies.
*   Our [designers folder](https://github.com/google/vizier/tree/main/vizier/_src/algorithms/designers) contains examples of designers.
*   Our [evolution folder](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/evolution) contains examples of creating evolutionary designers, such as [NSGA2](https://ieeexplore.ieee.org/document/996017/).
*   Our [designer testing routine](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/testing/test_runners.py) contains up-to-date examples on interacting with designers.